# Candy Distributor Shipping Analysis

This notebook loads the candy distributor dataset and performs exploratory data analysis (EDA) on shipping and supply-chain performance.


In [ ]:
# Install any required packages (uncomment if running in a fresh environment)
# %pip install pandas numpy matplotlib seaborn

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

sns.set(style="whitegrid")


## Load candy distributor dataset

Update `data_path` below to point to your actual dataset file (for example: `data/candy_distributor.csv`).


In [ ]:
# TODO: set this to the correct relative or absolute path of your candy distributor dataset

data_path_candy_sales = "dataset/Candy_Sales.csv"
data_path_candy_factories = "dataset/Candy_Factories.csv"  
data_path_candy_products = "dataset/Candy_Products.csv"  
data_path_candy_targets = "dataset/Candy_Targets.csv"  
data_path_distributor_data_dictionary = "dataset/candy_distributor_data_dictionary.csv"
data_path_uszips="dataset/uszips.csv"

if not os.path.exists(data_path):
    raise FileNotFoundError(
        f"Dataset not found at '{data_path}'. Please update 'data_path' to the correct location."
    )

# Read the dataset
df_candy_sales = pd.read_csv(data_path_candy_sales)
df_candy_factories = pd.read_csv(data_path_candy_factories)
df_candy_products = pd.read_csv(data_path_candy_products)
df_candy_targets = pd.read_csv(data_path_candy_targets)
df_candy_distributor_data_dictionary = pd.read_csv(data_path_distributor_data_dictionary)
df_uszips = pd.read_csv(data_path_uszips)
# Quick peek at the data
print("Shape:", df.shape)
df_candysales.head()


Loading datasets...

DATASET SUMMARY
Sales Data: 10,194 rows × 18 columns
Factories: 5 factories
Products: 15 products
US ZIP Codes: 33,787 ZIP codes

Sales Data Preview:


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State/Province,Postal Code,Division,Region,Product ID,Product Name,Sales,Units,Gross Profit,Cost
0,282,US-2021-128055-CHO-TRI-54000,2021-03-31,2026-09-26,Standard Class,128055,United States,San Francisco,California,94122,Chocolate,Pacific,CHO-TRI-54000,Wonka Bar - Triple Dazzle Caramel,7.50,2,4.90,2.60
1,288,US-2021-128055-CHO-SCR-58000,2021-03-31,2026-09-26,Standard Class,128055,United States,San Francisco,California,94122,Chocolate,Pacific,CHO-SCR-58000,Wonka Bar -Scrumdiddlyumptious,7.20,2,5.00,2.20
2,1132,US-2021-138100-CHO-FUD-51000,2021-09-15,2027-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,CHO-FUD-51000,Wonka Bar - Fudge Mallows,7.20,2,4.80,2.40
3,1133,US-2021-138100-CHO-MIL-31000,2021-09-15,2027-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,CHO-MIL-31000,Wonka Bar - Milk Chocolate,9.75,3,6.33,3.42
4,3396,US-2022-121391-CHO-MIL-31000,2022-10-04,2028-03-29,First Class,121391,United States,San Francisco,California,94109,Chocolate,Pacific,CHO-MIL-31000,Wonka Bar - Milk Chocolate,6.50,2,4.22,2.28


In [ ]:
since unique count of customer id is 5044 , they dont have repeating customer as of this information only. 

SALES DATA DEMOGRAPHICS

1. Country/Region Distribution:
Country/Region
United States    9994
Canada            200
Name: count, dtype: int64

2. Division Distribution:
Division
Chocolate    9844
Other         310
Sugar          40
Name: count, dtype: int64

3. Unique Products:
Total unique products: 15

4. Customer Base:
Unique customers: 5,044
Total orders: 8,549
Average orders per customer: 1.69



## Initial data inspection

Run the following cells after successfully loading the dataset to understand its structure and basic statistics.


In [ ]:
# Column info
print("Columns:\n", df.columns.tolist())
print("\nData types:")
print(df.dtypes)

# Missing values summary
print("\nMissing values per column:")
print(df.isna().sum())


SALES DATA STRUCTURE

Column Names:
['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Country/Region', 'City', 'State/Province', 'Postal Code', 'Division', 'Region', 'Product ID', 'Product Name', 'Sales', 'Units', 'Gross Profit', 'Cost']

Data Types:
Row ID              int64
Order ID           object
Order Date         object
Ship Date          object
Ship Mode          object
Customer ID         int64
Country/Region     object
City               object
State/Province     object
Postal Code        object
Division           object
Region             object
Product ID         object
Product Name       object
Sales             float64
Units               int64
Gross Profit      float64
Cost              float64
dtype: object

Missing Values Summary:
✓ No missing values found in sales data



In [ ]:
# Basic descriptive statistics for numeric columns
df.describe().T


NUMERIC COLUMNS SUMMARY STATISTICS


,count,mean,std,min,25%,50%,75%,max
Row ID,10194.0,5097.500000,2942.898656,1.00,2549.25,5097.50,7645.75,10194.0
Customer ID,10194.0,134468.961154,20231.483007,100006.00,117212.00,133550.00,152051.00,192314.0
Sales,10194.0,13.908537,11.341020,1.25,7.20,10.80,18.00,260.0
Units,10194.0,3.791838,2.228317,1.00,2.00,3.00,5.00,14.0
Gross Profit,10194.0,9.166451,6.643740,0.25,4.90,7.47,12.25,130.0
Cost,10194.0,4.742087,5.061647,0.60,2.40,3.60,5.70,130.0


1. What are the most efficient factory to customer shipping routes?

In [ ]:
df_candy_factories.head()

In [ ]:
df_candy_factories['Factory'].unique

FACTORY INFORMATION

Total Factories: 5

Factory Names:
["Lot's O' Nuts" "Wicked Choccy's" 'Sugar Shack' 'Secret Factory'
 'The Other Factory']

Factory Details:


,Factory,Latitude,Longitude
0,Lot's O' Nuts,32.881893,-111.768036
1,Wicked Choccy's,32.076176,-81.088371
2,Sugar Shack,48.119140,-96.181150
3,Secret Factory,41.446333,-90.565487
4,The Other Factory,35.117500,-89.971107


In [ ]:
df_candysales.head()

US ZIP Codes Dataset Structure:
Shape: (33787, 18)

Columns: ['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'zcta', 'parent_zcta', 'population', 'density', 'county_fips', 'county_name', 'county_weights', 'county_names_all', 'county_fips_all', 'imprecise', 'military', 'timezone']


,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16834.0,100.9,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37642.0,479.2,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,49075.0,551.7,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5590.0,48.7,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25542.0,265.7,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [ ]:
df_uszips.columns

Standardizing postal codes...
Merging sales data with ZIP code coordinates...

✓ Merge complete!
  - Total records: 10,194
  - Records with coordinates: 9,994
  - Records missing coordinates: 200 (1.96%)

Sales Data with Customer Coordinates:


,Order ID,Customer ID,Postal Code,City,State/Province,cust_lat,cust_lng
0,US-2021-128055-CHO-TRI-54000,128055,94122,San Francisco,California,37.76113,-122.48433
1,US-2021-128055-CHO-SCR-58000,128055,94122,San Francisco,California,37.76113,-122.48433
2,US-2021-138100-CHO-FUD-51000,138100,10011,New York City,New York,40.74173,-74.00037
3,US-2021-138100-CHO-MIL-31000,138100,10011,New York City,New York,40.74173,-74.00037
4,US-2022-121391-CHO-MIL-31000,121391,94109,San Francisco,California,37.79334,-122.42138


As of now using the uszip dataset we were able to find out the lat and longitude of the customers address. Now , we need to find efficient routes. We will do that using the lat and lon we have of both the factories and customer location. 

Analysis Metric,                  Calculation,                                What it tells you
Shipping Efficiency,              Cost / Distance,                            How much you are paying for every mile traveled. Lower is better.
Route Optimization,               Actual Factory vs Closest Factory,         Are you shipping from the nearest possible location?
Profit Density,                    Gross Profit / Distance,                Which routes are actually worth the logistics headache?

In [ ]:
from haversine import haversine, Unit

def calculate_distance(row, factory_lat, factory_long):
    customer_loc = (row['cust_lat'], row['cust_lng'])
    factory_loc = (factory_lat, factory_long)
    return haversine(customer_loc, factory_loc, unit=Unit.MILES)

for _, row in df_candy_factories.iterrows():
    factory_name = row['Factory']
    factory_lat = row['Latitude']
    factory_lng = row['Longitude']
    #col_name = f"dist_to_{factory_name.lower().replace(' ', '_').replace("'", '')}"
    #col_name = f"dist_to_{factory_name.lower().replace(' ', '_').replace(\"'\", '')}"
# Dynamically update the distance columns in sales_geo for each factory

# Remove all existing columns in sales_geo that start with 'dist_to_'
for col in list(sales_geo.columns):
    if col.startswith('dist_to_'):
        sales_geo = sales_geo.drop(columns=col)

# Add a new distance column for every factory, based on coordinates from df_candy_factories
for _, factory_row in df_candy_factories.iterrows():
    factory_name = factory_row['Factory']
    factory_lat = factory_row['Latitude']
    factory_lng = factory_row['Longitude']
    # Fix: do NOT use backslashes in f-string. Precompute string instead.
    safe_name = factory_name.lower().replace(' ', '_').replace("'", '')
    col_name = f"dist_to_{safe_name}"
    sales_geo[col_name] = sales_geo.apply(
    lambda cust_row: calculate_distance(cust_row, factory_lat, factory_lng), axis=1
    )
    








Calculating distances from each factory to customer locations...
Processing 5 factories × 10,194 customer locations...
  ✓ Calculated distances to Lot's O' Nuts
  ✓ Calculated distances to Wicked Choccy's
  ✓ Calculated distances to Sugar Shack
  ✓ Calculated distances to Secret Factory
  ✓ Calculated distances to The Other Factory

✓ Distance calculation complete!
  - Created 5 distance columns
  - Distance columns: ['dist_to_lots_o_nuts', 'dist_to_wicked_choccys', 'dist_to_sugar_shack', 'dist_to_secret_factory', 'dist_to_the_other_factory']


In [ ]:
sales_geo.head(1)

Sample of distance calculations:


,Order ID,Customer ID,City,State/Province,dist_to_lots_o_nuts,dist_to_wicked_choccys,dist_to_sugar_shack,dist_to_secret_factory,dist_to_the_other_factory
0,US-2021-128055-CHO-TRI-54000,128055,San Francisco,California,691.168202,2358.514641,1500.115375,1708.285735,1807.343000
1,US-2021-128055-CHO-SCR-58000,128055,San Francisco,California,691.168202,2358.514641,1500.115375,1708.285735,1807.343000
2,US-2021-138100-CHO-FUD-51000,138100,New York City,New York,2140.827453,716.223347,1201.551721,862.615751,951.311969
3,US-2021-138100-CHO-MIL-31000,138100,New York City,New York,2140.827453,716.223347,1201.551721,862.615751,951.311969
4,US-2022-121391-CHO-MIL-31000,121391,San Francisco,California,689.062587,2354.962139,1496.029939,1704.317592,1803.752682


In [ ]:
#based on the distance the most efficient routes are as such 
factory_cols = ['dist_to_lots_o_nuts', 'dist_to_wicked_choccys', 'dist_to_sugar_shack', 'dist_to_secret_factory','dist_to_the_other_factory']
#sales_geo[['Customer ID','dist_to_lots_o_nuts','dist_to_wicked_choccys','dist_to_sugar_shack','dist_to_secret_factory','dist_to_the_other_factory']][:5]

✓ All 5 distance columns found
  Columns: ['dist_to_lots_o_nuts', 'dist_to_wicked_choccys', 'dist_to_sugar_shack', 'dist_to_secret_factory', 'dist_to_the_other_factory']


In [ ]:
sales_geo['closest_factory'] = sales_geo[factory_cols].idxmin(axis=1)
sales_geo.head()

Identifying closest factory for each order...
✓ Closest factory identified for 10,194 orders

Closest Factory Distribution:
closest_factory
Wicked Choccys       3400
Lots O Nuts          3217
Secret Factory       1745
The Other Factory    1536
Sugar Shack            96
Name: count, dtype: int64

Sample Results:


/var/folders/t8/vh7brvtx5ln20x3lyxhn_hhw0000gp/T/ipykernel_77791/1357800965.py:4: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  sales_geo['closest_factory'] = sales_geo[factory_cols].idxmin(axis=1)


,Order ID,City,State/Province,closest_factory,dist_to_lots_o_nuts,dist_to_wicked_choccys,dist_to_sugar_shack
0,US-2021-128055-CHO-TRI-54000,San Francisco,California,Lots O Nuts,691.168202,2358.514641,1500.115375


In [125]:
# Extract the distance value to the closest factory for each row
# Method: Use advanced indexing to look up values based on column names
# This replaces the deprecated .lookup() method

print("Extracting distances to closest factory...")

# Get the column index for each closest_factory column name
col_indices = sales_geo[factory_cols].columns.get_indexer(sales_geo['closest_factory'])

# Extract values using row indices and column indices
distance_to_closest = sales_geo[factory_cols].values[
    np.arange(len(sales_geo)), 
    col_indices
]

# Add as a new column
sales_geo['distance_to_closest'] = distance_to_closest

print(f"✓ Distance to closest factory calculated")
print(f"\nDistance Statistics (miles):")
print(sales_geo['distance_to_closest'].describe())

# Display sample
print("\nSample Results:")
sales_geo[['Order ID', 'City', 'closest_factory', 'distance_to_closest']].head()

Extracting distances to closest factory...
✓ Distance to closest factory calculated

Distance Statistics (miles):
count    9994.000000
mean      963.285201
std       513.026089
min         7.497484
25%       512.978594
50%       878.187418
75%      1562.002625
max      1896.012356
Name: distance_to_closest, dtype: float64

Sample Results:


,Order ID,City,closest_factory,distance_to_closest
0,US-2021-128055-CHO-TRI-54000,San Francisco,Lots O Nuts,1807.343000
1,US-2021-128055-CHO-SCR-58000,San Francisco,Lots O Nuts,1807.343000
2,US-2021-138100-CHO-FUD-51000,New York City,Wicked Choccys,951.311969
3,US-2021-138100-CHO-MIL-31000,New York City,Wicked Choccys,951.311969
4,US-2022-121391-CHO-MIL-31000,San Francisco,Lots O Nuts,1803.752682


In [ ]:
sales_geo['profit_density'] = sales_geo['Gross Profit'] / distance_to_closest

# Display example output
sales_geo[['Gross Profit', 'closest_factory', 'profit_density']].head()

In [127]:
# Calculate profit density: Gross Profit per mile traveled
# This metric helps identify which routes generate the most value per unit of distance
sales_geo['profit_density'] = sales_geo['Gross Profit'] / sales_geo['distance_to_closest']

# Handle division by zero or negative distances (shouldn't occur, but safety check)
sales_geo['profit_density'] = sales_geo['profit_density'].replace([np.inf, -np.inf], np.nan)

print("="*60)
print("PROFIT DENSITY ANALYSIS")
print("="*60)
print("\nProfit Density Statistics ($ per mile):")
print(sales_geo['profit_density'].describe())

print("\nTop 10 Orders by Profit Density:")
top_profit_density = sales_geo.nlargest(10, 'profit_density')[
    ['Order ID', 'City', 'State/Province', 'closest_factory', 
     'Gross Profit', 'distance_to_closest', 'profit_density']
]
print(top_profit_density.to_string(index=False))

print("\nSample Results:")
sales_geo[['Order ID', 'Gross Profit', 'closest_factory', 'distance_to_closest', 'profit_density']].head()

PROFIT DENSITY ANALYSIS

Profit Density Statistics ($ per mile):
count    9994.000000
mean        0.016529
std         0.060635
min         0.000134
25%         0.005042
50%         0.008909
75%         0.016051
max         2.251422
Name: profit_density, dtype: float64

Top 10 Orders by Profit Density:
                    Order ID      City State/Province   closest_factory  Gross Profit  distance_to_closest  profit_density
US-2022-151547-CHO-MIL-31000  Bartlett      Tennessee The Other Factory         16.88             7.497484        2.251422
US-2024-128363-CHO-SCR-58000   Memphis      Tennessee The Other Factory         17.50            10.928693        1.601289
US-2024-128118-CHO-NUT-13000   Memphis      Tennessee The Other Factory         17.43            10.928693        1.594884
US-2022-134026-CHO-MIL-31000   Memphis      Tennessee The Other Factory         16.88            10.928693        1.544558
US-2022-153038-CHO-MIL-31000   Memphis      Tennessee The Other Factory         1

,Order ID,Gross Profit,closest_factory,distance_to_closest,profit_density
0,US-2021-128055-CHO-TRI-54000,4.90,Lots O Nuts,1807.343000,0.002711
1,US-2021-128055-CHO-SCR-58000,5.00,Lots O Nuts,1807.343000,0.002766
2,US-2021-138100-CHO-FUD-51000,4.80,Wicked Choccys,951.311969,0.005046
3,US-2021-138100-CHO-MIL-31000,6.33,Wicked Choccys,951.311969,0.006654
4,US-2022-121391-CHO-MIL-31000,4.22,Lots O Nuts,1803.752682,0.002340


In [ ]:
sales_geo[['closest_factory','profit_density','Gross Profit','is_profitable_route','break_even_distance']]

In [ ]:
sales_geo[sales_geo['is_profitable_route']==False]['closest_factory'].value_counts()


Using shipping rate: $0.0100 per mile

Calculating break-even metrics...
ROUTE PROFITABILITY ANALYSIS

Total Orders: 10,194
Profitable Routes: 4,542 (44.56%)
Unprofitable Routes: 5,652 (55.44%)

Break-Even Distance Statistics (miles):
count    10194.000000
mean       916.645085
std        664.373982
min         25.000000
25%        490.000000
50%        747.000000
75%       1225.000000
max      13000.000000
Name: break_even_distance, dtype: float64

Distance Margin Statistics (miles):
count     9994.000000
mean       -47.653522
std        836.321146
min      -1845.026211
25%       -560.229034
50%        -89.773568
75%        375.454139
max      12121.812582
Name: distance_margin, dtype: float64

Sample Results:


,Order ID,closest_factory,Gross Profit,distance_to_closest,break_even_distance,distance_margin,is_profitable_route
0,US-2021-128055-CHO-TRI-54000,Lots O Nuts,4.90,1807.343000,490.0,-1317.343000,False
1,US-2021-128055-CHO-SCR-58000,Lots O Nuts,5.00,1807.343000,500.0,-1307.343000,False
2,US-2021-138100-CHO-FUD-51000,Wicked Choccys,4.80,951.311969,480.0,-471.311969,False
3,US-2021-138100-CHO-MIL-31000,Wicked Choccys,6.33,951.311969,633.0,-318.311969,False
4,US-2022-121391-CHO-MIL-31000,Lots O Nuts,4.22,1803.752682,422.0,-1381.752682,False
5,US-2023-103982-CHO-SCR-58000,The Other Factory,2.50,547.612682,250.0,-297.612682,False
6,US-2023-103982-CHO-NUT-13000,The Other Factory,7.47,547.612682,747.0,199.387318,True
7,US-2023-103982-CHO-FUD-51000,The Other Factory,16.80,547.612682,1680.0,1132.387318,True
8,US-2023-103982-CHO-MIL-31000,The Other Factory,6.33,547.612682,633.0,85.387318,True
9,US-2024-147039-CHO-NUT-13000,Sugar Shack,7.47,700.013973,747.0,46.986027,True


UNPROFITABLE ROUTES BY FACTORY

Total Unprofitable Orders: 5,652

Unprofitable Routes by Closest Factory:
closest_factory
Lots O Nuts          2780
Wicked Choccys       1876
Secret Factory        514
The Other Factory     238
Sugar Shack            44
Name: count, dtype: int64

Percentage of Unprofitable Routes by Factory:
                   total_orders  profitable  unprofitable  unprofitable_pct
closest_factory                                                            
Lots O Nuts                3217         437          2780         86.415915
Wicked Choccys             3400        1524          1876         55.176471
Sugar Shack                  96          52            44         45.833333
Secret Factory             1745        1231           514         29.455587
The Other Factory          1536        1298           238         15.494792


MOST EFFICIENT ROUTES SUMMARY

Top 20 Most Efficient Routes (minimum 5 orders):
          Factory           City       State  Order_Count  Total_Profit  Avg_Distance  Avg_Profit_Density  Profitable_Orders  Avg_Distance_Margin  Profitability_Pct
The Other Factory      Southaven Mississippi            7         59.92     10.712919            0.799035                  7           845.287081         100.000000
The Other Factory        Memphis   Tennessee           30        256.24     10.928693            0.781551                 30           843.204640         100.000000
The Other Factory      Jonesboro    Arkansas           11        122.98     64.819917            0.172478                 11          1053.180083         100.000000
The Other Factory        Jackson   Tennessee           10         96.40     70.982791            0.135808                 10           893.017209         100.000000
The Other Factory       Florence     Alabama            5         51.27    132.385277          